In [83]:
# Import Libraries # 
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib as plt

# Load spreadsheet # 
IOlog = pd.read_excel('IO log sample.xlsx')
time_interval = IOlog.shape[0]
print(time_interval)
# Size sheet
data2 = IOlog.to_numpy()
data = np.flip(data2, axis=0)
size = data.size
col = int(size/len(data))
row = len(data)
print(col)
print(row)
rowtrun = row - 11
print(str(data[1:5,3]))

# Time converter
def convert_time(time_strings):
    time_format = "%m-%d-%Y %H:%M:%S"
    base_time = datetime.strptime(time_strings[0], time_format)
    
    time_in_minutes = [(datetime.strptime(t, time_format) - base_time).total_seconds() / 60 for t in time_strings]
    return time_in_minutes

# Initialise controllers and instruments - PRESENTLY HARDCODED, NEED TO MOVE TO EXCEL DATAFRAME!
controllers = {"LIC101" : [0,0,0,0,0], "LIC108" : [0,0,0,0,0], "FIC103": [0,0,0,0,0], "FIC108": [0,0,0,0,0]}
controllers["LIC108"][0] = 1
print(controllers)
pumps = {"J100": [0,0], "J101a": [0,0], "J101b_": [0,0], "J102": [0,0], "J103": [0,0], "J104": [0,0], "J105": [0,0], "J106": [0,0]}
print(pumps)

259
11
259
['Acknowledge of LL condition on object FT109 has been requested.'
 'Acknowledge of L condition on object FT109 has been requested.'
 'Acknowledge of L condition on object FT109 has been requested.'
 'Acknowledge of LL condition on object FT109 has been requested.']
{'LIC101': [0, 0, 0, 0, 0], 'LIC108': [1, 0, 0, 0, 0], 'FIC103': [0, 0, 0, 0, 0], 'FIC108': [0, 0, 0, 0, 0]}
{'J100': [0, 0], 'J101a': [0, 0], 'J101b_': [0, 0], 'J102': [0, 0], 'J103': [0, 0], 'J104': [0, 0], 'J105': [0, 0], 'J106': [0, 0]}


In [108]:
ManPumpSpeedchanges = []
Pchanges = []
Ichanges = []
pumpmodechanges = []
controllerSPchanges = []
ManPumpSpeedchangesT = []
PchangesT = []
IchangesT = []
pumpmodechangesT = []
controllerSPchangesT = []

for i in range(2,len(data)-1):

# Pump change detection
    timechange = str(data[i,0])
    item = str(data[i,1])
    inputlog = str(data[i,3])
    
    if inputlog[0:10] == "HSICmd.Man":
        if inputlog[10:12] == "SP":
            speedchange = inputlog[12:]
            speeds = speedchange.split()
            speeds.append(timechange)
            b = speeds[2]
            pumps[item][0] = int(b)
            speeds.pop(1)
            ManPumpSpeedchanges.append(speeds)
            ManPumpSpeedchangesT.append(timechange)
            print(pumps)

        # Detect pump mode - TO DO: CHANGE OUTPUT DEPENDING ON REQUIREMENT
        elif inputlog[7:10] == "Man":
            pumps[item][1] = int(0)
            pumpmodechanges.append(["Man",timechange])
            pumpmodechangesT.append(timechange)
    
    elif inputlog == "HSICmd.Auto False -> True ":
        pumps[item][1] = int(1)
        pumpmodechanges.append(["Auto",timechange])
        pumpmodechangesT.append(timechange)
        print(pumps)
                
    elif inputlog[0:9] == "HSICmd.SP":
        setptchange = inputlog[18:]
        setpts = setptchange.split()
        print(setpts)
        sp1 = setpts[0]
        sp2 = setpts[2]
        controllers[item][4] = int(sp2)
        controllerSPchanges.append([item,sp2,timechange])
        controllerSPchangesT.append(timechange)
        print(controllers)
    
    # Controller changes
    elif inputlog[0:14] == "InteractionPar":
        if inputlog[20:34] == "ControllerType":
            modechange = inputlog[35:]
            modes = modechange.split()
            mode1 = modes[0]
            mode2 = modes[2]
            controllers[item][3] = mode2
        
        # P changes
        elif inputlog[15:24] == "Main.Gain":
            gainchange = inputlog[25:]
            gains = gainchange.split()
            P2 = gains[2]
            gains.append(timechange)
            controllers[item][0] = P2
            gains.pop(1)
            gains.insert(0,item)
            mode = controllers[item][3]
            gains.insert(1,mode)
            print(controllers)
            Pchanges.append(gains)
            PchangesT.append(timechange)
            
        # I changes
        elif inputlog[15:22] == "Main.Ti":
            intchange = inputlog[23:]
            ints = intchange.split()
            Ti1 = ints[0]
            Ti2 = ints[2]
            controllers[item][1] = Ti2
            print(controllers)
            ints.append(timechange)
            ints.pop(1)
            ints.insert(0,item)
            mode = controllers[item][3]
            ints.insert(1,mode)
            print(controllers)
            Ichanges.append(ints)
            IchangesT.append(timechange)
           
print(ManPumpSpeedchanges,ManPumpSpeedchangesT)
print(pumpmodechanges,pumpmodechangesT)
print(Pchanges,PchangesT)
print(Ichanges,IchangesT)
print(controllerSPchanges,controllerSPchangesT)

{'LIC101': ['0.1', '20', 0, '1', 0], 'LIC108': ['2.5', '0', 0, '1', 35], 'FIC103': ['0.01', '20', 0, '4', 0], 'FIC108': ['0.1', '20', 0, '4', 150]}
{'LIC101': ['0.1', '20', 0, '4', 0], 'LIC108': ['2.5', '0', 0, '1', 35], 'FIC103': ['0.01', '20', 0, '4', 0], 'FIC108': ['0.1', '20', 0, '4', 150]}
{'LIC101': ['0.1', '20', 0, '4', 0], 'LIC108': ['2.5', '0', 0, '1', 35], 'FIC103': ['0.01', '20', 0, '4', 0], 'FIC108': ['0.1', '20', 0, '4', 150]}
{'LIC101': ['0.1', '20', 0, '4', 0], 'LIC108': ['2.5', '0', 0, '1', 35], 'FIC103': ['0.1', '20', 0, '4', 0], 'FIC108': ['0.1', '20', 0, '4', 150]}
{'LIC101': ['0.1', '20', 0, '4', 0], 'LIC108': ['2.5', '0', 0, '1', 35], 'FIC103': ['0.1', '20', 0, '4', 0], 'FIC108': ['0.1', '20', 0, '4', 150]}
{'LIC101': ['0.1', '20', 0, '4', 0], 'LIC108': ['2.5', '0', 0, '1', 35], 'FIC103': ['0.1', '20', 0, '4', 0], 'FIC108': ['0.1', '20', 0, '4', 150]}
{'LIC101': ['0.1', '20', 0, '4', 0], 'LIC108': ['2.5', '0', 0, '1', 35], 'FIC103': ['0.1', '20', 0, '4', 0], 'FIC10